# 🚀 IDDAW Model Training on Google Colab

This notebook trains your IDDAW segmentation models on **FREE GPU**!

## Before Starting:
1. **Enable GPU**: Runtime → Change runtime type → Hardware accelerator → **GPU (T4)**
2. **Upload your project** to Google Drive OR use the upload method below

---

## Step 1: Check GPU

In [ ]:
import torch
import sys

print("="*70)
print("GPU Check")
print("="*70)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("\n✅ GPU is ready!")
else:
    print("\n⚠️ WARNING: No GPU detected!")
    print("Go to: Runtime → Change runtime type → GPU")
    print("Then restart this notebook.")

## Step 2: Mount Google Drive (Option A)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# List your Drive contents
!ls /content/drive/MyDrive/

## Step 2 Alternative: Upload Project as ZIP (Option B)

If you haven't uploaded to Drive, run this cell to upload directly:

In [ ]:
from google.colab import files
import zipfile
import os

print("Upload your iddaw_project.zip file...")
uploaded = files.upload()

# Extract
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\nExtracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/iddaw')
        print("✅ Extraction complete!")
        break

## Step 3: Setup Project Directory

In [ ]:
import os

# Choose one:
# Option A: From Google Drive
project_path = '/content/drive/MyDrive/iddaw/pro'  # Adjust this path!

# Option B: From uploaded zip
# project_path = '/content/iddaw'

os.chdir(project_path)
print(f"Working directory: {os.getcwd()}")
print("\nProject contents:")
!ls -la

## Step 4: Install Dependencies

In [ ]:
!pip install -q opencv-python-headless pillow numpy
print("✅ Dependencies installed!")

## Step 5: Verify Dataset

In [ ]:
import os

print("Checking dataset...")
print("\nIDDAW directory:")
!ls -la IDDAW/

print("\nSplits directory:")
!ls -la project/splits/

print("\nChecking train.csv:")
!head -5 project/splits/train.csv

## 🚀 Training Options

Choose which model to train:
- **Option 1**: NIR FastSCNN (FASTEST - 30-45 min)
- **Option 2**: RGB FastSCNN (Fast - 1-1.5 hours)
- **Option 3**: Mid-fusion (BEST - 2-3 hours)
- **Option 4**: Train all models

---

## Option 1: Train NIR FastSCNN (FASTEST ⚡⚡⚡)

**Time**: ~30-45 minutes  
**Accuracy**: Good (mIoU ~0.55-0.60)

In [ ]:
!python -m project.train_improved \
    --modality nir \
    --backbone fastscnn \
    --epochs 50 \
    --batch_size 16 \
    --lr 1e-3 \
    --amp \
    --scheduler cosine \
    --class_weights \
    --out_dir project/ckpts

## Option 2: Train RGB FastSCNN (Fast ⚡⚡)

**Time**: ~1-1.5 hours  
**Accuracy**: Better (mIoU ~0.58-0.63)

In [ ]:
!python -m project.train_improved \
    --modality rgb \
    --backbone fastscnn \
    --epochs 50 \
    --batch_size 16 \
    --lr 1e-3 \
    --amp \
    --scheduler cosine \
    --class_weights \
    --out_dir project/ckpts

## Option 3: Train Mid-Fusion (BEST ⭐⭐⭐⭐⭐)

**Time**: ~2-3 hours  
**Accuracy**: Best (mIoU ~0.60-0.70)

In [ ]:
!python -m project.train_improved \
    --modality mid \
    --backbone mbv3 \
    --epochs 50 \
    --batch_size 8 \
    --lr 1e-3 \
    --amp \
    --scheduler cosine \
    --class_weights \
    --out_dir project/ckpts

## Option 4: Train All Models (3-4 hours total)

In [ ]:
models = [
    {"name": "NIR FastSCNN", "modality": "nir", "backbone": "fastscnn", "batch": 16},
    {"name": "RGB FastSCNN", "modality": "rgb", "backbone": "fastscnn", "batch": 16},
    {"name": "Mid-Fusion", "modality": "mid", "backbone": "mbv3", "batch": 8},
]

for i, model in enumerate(models, 1):
    print("\n" + "="*70)
    print(f"Training {i}/{len(models)}: {model['name']}")
    print("="*70 + "\n")
    
    !python -m project.train_improved \
        --modality {model['modality']} \
        --backbone {model['backbone']} \
        --epochs 50 \
        --batch_size {model['batch']} \
        --lr 1e-3 \
        --amp \
        --scheduler cosine \
        --class_weights \
        --out_dir project/ckpts

print("\n" + "="*70)
print("✅ All models trained!")
print("="*70)

## Step 6: Check Training Results

In [ ]:
import torch
from pathlib import Path

ckpt_dir = Path('project/ckpts')
print("Trained models:")
print("="*70)

for ckpt_file in sorted(ckpt_dir.glob('best_*.pt')):
    ckpt = torch.load(ckpt_file, map_location='cpu')
    miou = ckpt.get('miou', 0)
    safe_miou = ckpt.get('safety', {}).get('safe_mIoU', 0)
    epoch = ckpt.get('epoch', '?')
    
    print(f"\n{ckpt_file.name}")
    print(f"  mIoU: {miou:.3f}")
    print(f"  Safe mIoU: {safe_miou:.3f}")
    print(f"  Epoch: {epoch}")

print("\n" + "="*70)

## Step 7: Download Trained Models

In [ ]:
import shutil
from google.colab import files

# Create zip of checkpoints
print("Creating zip file...")
shutil.make_archive('trained_models', 'zip', 'project/ckpts')
print("✅ Zip created!")

# Download
print("\nDownloading... (this may take a minute)")
files.download('trained_models.zip')
print("✅ Download complete!")

## Step 8: Save to Google Drive (Backup)

In [ ]:
import shutil
from datetime import datetime

# Create backup in Drive
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
backup_path = f'/content/drive/MyDrive/iddaw_models_{timestamp}'

print(f"Backing up to: {backup_path}")
shutil.copytree('project/ckpts', backup_path)
print("✅ Backup complete!")
print(f"\nYour models are saved in Google Drive at:")
print(backup_path)

## 🎉 Training Complete!

### Next Steps:

1. **Download** `trained_models.zip` (already done in Step 7)
2. **Extract** on your local machine:
   ```powershell
   cd D:\iddaw\pro
   Expand-Archive -Path trained_models.zip -DestinationPath . -Force
   ```
3. **Test** the new models:
   ```powershell
   cd D:\iddaw\pro\project
   python backend_with_models.py
   ```
4. **Upload images** through web interface and see improved results!

---

### Training Summary:

- ✅ Models trained on FREE GPU
- ✅ 50 epochs with learning rate scheduling
- ✅ Class weights for better accuracy
- ✅ Models saved and backed up

**Your segmentation accuracy should be significantly improved!** 🚀

---

## 🔥 Pro Tips

### Keep Colab Alive

Run this cell to prevent disconnection during long training:

In [ ]:
import time
from IPython.display import clear_output

print("Keep-alive started. This will print every 5 minutes.")
print("Stop this cell when training is complete.\n")

counter = 0
while True:
    time.sleep(300)  # 5 minutes
    counter += 1
    clear_output(wait=True)
    print(f"Still alive... ({counter * 5} minutes elapsed)")
    print("Training in progress...")

### Resume Training

If training stops, resume with:

In [ ]:
# Resume mid-fusion training
!python -m project.train_improved \
    --modality mid \
    --epochs 50 \
    --batch_size 8 \
    --amp \
    --scheduler cosine \
    --class_weights \
    --resume  # This flag resumes from last checkpoint